# Face and Mask

In [1]:
import cv2
import matplotlib.pyplot as plt
import helpers
import numpy as np
import tensorflow as tf
import glob
import random
from retinaface import RetinaFace
from PIL import Image
from od_metrics.BoundingBox import BoundingBox
from od_metrics.BoundingBoxes import BoundingBoxes
from od_metrics.utils import BBFormat, BBType
from od_metrics.Evaluator import Evaluator
use_pickle = False

## Load Kaggle Dataset
Kaggle dataset has 853 images

In [2]:
# load images and associated faces
# this takes a second since we load with cv2
imgs_with_labels = helpers.load_kaggle_863('../kaggle_dataset_863')
imgs_with_labels = helpers.convert_kaggle_863_for_metrics(imgs_with_labels)

## Facial Recognition
We use retinaface to detect facial features

In [3]:
# Initialize facial detection
detector = RetinaFace(quality = "normal")

model[normal quality] init ..
model success !


In [4]:
dl_detection_results = {}
dl_detection_results['retinaface_mobilenet_alt'] = imgs_with_labels['bboxes'].clone()
detections = dl_detection_results['retinaface_mobilenet_alt']

In [7]:
#Detect faces and crop them, add the cropped Faces into cropImages

cropImages = []
cnt = 0
for img_name, img_data in imgs_with_labels['raw_data'].items():
    img = img_data['img']
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # using default threshold
    faces = detector.predict(rgb_img.astype(np.float32), threshold=0.7)
    
    #Crop faces and add them to cropImages
    img = Image.fromarray(rgb_img)
    cropFaces = []
    for face in faces:
        cropFaces.append(img.crop((face['x1'], face['y1'], face['x2'], face['y2'])))
        
    cropImages.append(cropFaces)
    
    cnt += 1
    if cnt%25 == 0: print(cnt)

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850


## Mask Detection
we will now pass cropImages through the mask classifier

In [26]:
dim = 64

# I am passing in PIL images, this converts it to PIL
def resize_img(pic):
    image_array = tf.keras.preprocessing.image.img_to_array(pic)
    img = tf.convert_to_tensor(image_array, dtype = 'uint8')
    img = tf.image.resize(img, [dim, dim])
    img = img/127.5-1
    return img

In [27]:
# The neural network

def conv_model(num_blocks, rate, dim):
    
    input = tf.keras.layers.Input(shape=(dim,dim,3))
    
    x = tf.keras.layers.Conv2D(16, (16,16), strides=1)(input)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
        
    for i in range(num_blocks-1):
        x = tf.keras.layers.Conv2D(8, (8,8), strides=1)(x)
        x = tf.keras.layers.Dropout(rate)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        
    x = tf.keras.layers.Conv2D(filters=2, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)
    
    model = tf.keras.Model(inputs=input, outputs=predictions)
    print(model.summary())
    
    return model

In [28]:
num_blocks = 3
dropout = .1
model = conv_model(num_blocks,.1, dim)
model.load_weights('mask_classification_model_3_50.h5')
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 49, 49, 16)        12304     
_________________________________________________________________
dropout (Dropout)            (None, 49, 49, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 49, 49, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 8)         8200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 42, 8)         0     

In [29]:
# Prediction function, passes in PIL image
def pred(pic):
    im = resize_img(pic) # Convert and resize PIL image to 64x64 tensor
    vals = model.predict(np.expand_dims(im,axis=0))
    print(f"Mask Val: {vals[0][0]}, Non-mask Val: {vals[0][1]}")
    prediction = np.argmax(vals,axis=1)
    print(f"Image predicted as {prediction}")
    return prediction